In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho correto para a raiz do projeto Kedro
project_path = r"c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project"

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

# Acessar os parâmetros
params = context.params

# Listar o catálogo
catalog.list()

[09/15/24 19:26:41] INFO     Using                                                                  ]8;id=368227;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=169814;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


[09/15/24 19:26:43] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=778537;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=276992;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         


[
    'decks_json_partitioned',
    'decks_txt_partitioned',
    'sampled_decks',
    'players',
    'players_with_decks',
    'matches_df',
    'features_df',
    'selected_features_df',
    'train_features',
    'test_features',
    'train_target',
    'test_target',
    'model_pkl',
    'parameters',
    'params:global',
    'params:global.run_date',
    'params:global.run_date.day',
    'params:global.run_date.month',
    'params:global.run_date.year',
    'params:global.user',
    'params:global.user.project_path',
    'params:preprocessing',
    'params:preprocessing.webscraper',
    'params:preprocessing.webscraper.zip_url',
    'params:preprocessing.webscraper.zip_folder',
    'params:preprocessing.webscraper.log_folder',
    'params:preprocessing.webscraper.deck_cards',
    'params:preprocessing.webscraper.sample_size_ratio',
    'params:simulation',
    'params:simulation.n_players',
    'params:simulation.matches_per_player',
    'params:simulation.hand_size_stop',
    'par

In [2]:
from typing import List, Dict
from classes.deck import Deck
from classes.player import Player
from classes.player_tracker import PlayerTracker
from src.mtg_project.pipelines.utils import setup_logger
import pandas as pd
import numpy as np
from faker import Faker
import random

In [3]:
def create_players(n_players: int):
    """
    Cria uma lista de objetos Player com nomes aleatórios.

    Args:
        n_players (int): Número de jogadores a serem criados.

    Returns:
        List[Player]: Lista de objetos Player com nomes gerados aleatoriamente.
    """
    # Inicializando o gerador de dados falsos Faker
    fake = Faker()
    
    # Gerando uma lista de nomes aleatórios usando o Faker
    player_names = [fake.first_name() + " " + fake.last_name() for _ in range(n_players)]
    
    # Criando uma lista de objetos Player a partir dos nomes gerados
    players = [Player(name) for name in player_names]

    # Retornando a lista de objetos Player
    return players

n_players = catalog.load("params:simulation.n_players")
players = create_players(n_players)
catalog.save("players", players)

[09/15/24 19:26:44] INFO     Loading data from params:simulation.n_players (MemoryDataset)...   ]8;id=107749;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=27348;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Saving data to players (PickleDataset)...                          ]8;id=728061;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=753561;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [4]:
import random
import os
from typing import List, Dict

def assign_decks_to_players(
        players: List[Player], 
        sampled_decks: Dict[str, str],
        log_folder: str) -> List[Player]:
    """
    Função para atribuir decks aleatórios a cada player na lista de players.

    A função tentará atribuir um deck a cada player chamando o método assign_deck().
    Caso ocorra algum erro na atribuição, tentará com outro deck disponível.

    Args:
        players (list): Lista de objetos Player.
        sampled_decks (dict): Dicionário com os nomes e caminhos dos decks.
        log_folder (str): Caminho da pasta para salvar o log.

    Returns:
        List[Player]: Lista de objetos Player com decks atribuídos.
    """
    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_assignment.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("validate_decks", log_filepath)
    
    # Log de início da validação
    logger.info("Validating decks...")

    # Convertemos as chaves do dicionário para uma lista de nomes de decks disponíveis
    available_decks = list(sampled_decks.keys())
    
    for player in players:
        assigned = False
        while not assigned and available_decks:
            try:
                # Seleciona um deck aleatório da lista de decks disponíveis
                deck_name = random.choice(available_decks)

                # Obter o caminho completo do deck a partir do dicionário sampled_decks
                deck_path = sampled_decks[deck_name]

                # Cria um novo objeto Deck
                deck = Deck()

                # Carrega o deck a partir do arquivo .txt no caminho obtido
                deck.load_deck_from_txt(deck_path)

                # Atribui o deck ao player
                player.assign_deck(deck)
                logger.info(f"Deck '{deck_name}' assigned to player '{player.name}'")
                
                # Remove o deck da lista de decks disponíveis para evitar reutilização
                available_decks.remove(deck_name)

                assigned = True  # Deck atribuído com sucesso
            except Exception as e:
                # Em caso de erro, tenta outro deck
                logger.error(f"Failed to assign deck '{deck_name}' to player '{player.name}': {e}")
                continue
        
        
        # Se não houver mais decks disponíveis e não conseguir atribuir, lança um erro
        if not assigned:
            raise ValueError(f"No available decks left to assign to player '{player.name}'.")

    logger.info("Deck assignment process completed.")

    return players

players = catalog.load("players")
sampled_decks = catalog.load("sampled_decks")
log_folder = catalog.load("params:simulation.log_folder")
# players_with_decks = assign_decks_to_players(players, sampled_decks, log_folder)
# catalog.save("players_with_decks", players_with_decks)

                    INFO     Loading data from players (PickleDataset)...                       ]8;id=513801;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=372376;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from sampled_decks (PickleDataset)...                 ]8;id=416633;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=240999;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from params:simulation.log_folder (MemoryDataset)...  ]8;id=642475;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=878660;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [5]:
def simulate_player_matches(params: dict, players_with_decks: list) -> pd.DataFrame:
    """
    Simulates Magic: The Gathering matches for a list of players based on the provided simulation parameters.

    Parameters:
    -----------
    params : dict
        A dictionary containing the simulation parameters, including:
        - 'max_mulligans': Maximum number of mulligans allowed per player.
        - 'mulligan_prob': Probability of a player choosing to mulligan.
        - 'hand_size_stop': Minimum hand size at which the simulation will stop.
        - 'max_turns': Maximum number of turns per match.
        - 'extra_land_prob': Probability of playing an extra land during a turn.
        - 'matches_per_player': Number of matches to simulate per player.
        - 'log_folder': Folder path for logging the simulation process.
    
    players_with_decks : list
        A list of Player objects, each with an assigned deck to be used in the simulation.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the match data for all players across all matches and turns, including:
        - Player attributes at each turn.
        - Match number for each simulation.
    """
    
    # Atribuir os parâmetros
    max_mulligans = params["max_mulligans"]
    mulligan_prob = params["mulligan_prob"]
    hand_size_stop = params["hand_size_stop"]
    max_turns = params["max_turns"]
    extra_land_prob = params["extra_land_prob"]
    matches_per_player = params["matches_per_player"]
    log_folder = params["log_folder"]

    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'player_matches.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("player_matches", log_filepath)

    # Log de início da validação
    logger.info("Initiating simulations...")

    # Inicializa o tracker para armazenar os dados
    tracker = PlayerTracker()

    # Loop através dos jogadores e realizar as simulações de partidas
    for player in players_with_decks:
        for match in range(matches_per_player):
            # Simular várias partidas para o jogador
            player.play_a_match(tracker, 
                                max_mulligans, 
                                mulligan_prob, 
                                max_turns, 
                                hand_size_stop, 
                                extra_land_prob)

    # Obter os dados de todas as partidas e turnos
    matches_df = tracker.get_data()
    
    return matches_df

# Chamada da função
params = catalog.load("params:simulation")
players_with_decks = catalog.load("players_with_decks")
features_df = simulate_player_matches(params, players_with_decks)
catalog.save("matches_df",features_df)

                    INFO     Loading data from params:simulation (MemoryDataset)...             ]8;id=253664;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=482701;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from players_with_decks (PickleDataset)...            ]8;id=729915;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=417659;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    WARNING  Steven Guerrero couldn't play any spells.                                ]8;id=367715;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=309254;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Steven Guerrero couldn't play any spells.                                ]8;id=557474;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=744082;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Steven Guerrero couldn't play any spells.                                ]8;id=897137;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=321392;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

...

                    WARNING  Jessica Smith couldn't play any spells.                                  ]8;id=719880;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=156979;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

[09/15/24 19:26:45] WARNING  Jessica Smith couldn't play any spells.                                  ]8;id=221884;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=678203;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Jessica Smith couldn't play any spells.                                  ]8;id=596898;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=591026;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Jessica Smith couldn't play any spells.                                  ]8;id=135670;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=244129;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    INFO     Saving data to matches_df (ParquetDataset)...                      ]8;id=772146;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=558276;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [6]:
features_df

,name,deck_name,deck_colors,match,turn,mulligan_count,lands_played,spells_played,mana_pool,spent_mana,hand_size,library_size,graveyard_size,full_hand,full_graveyard
0,Steven Guerrero,elvish rage,{G},1,0,0,0,0,0,0,7,53,0,"Hand(7 cards: Karplusan Forest, Karplusan Fore...",Graveyard(0 cards: )
1,Steven Guerrero,elvish rage,{G},1,1,0,1,1,1,1.0,6,52,1,"Hand(6 cards: Karplusan Forest, Tribal Forcema...",Graveyard(1 cards: Elvish Pioneer)
2,Steven Guerrero,elvish rage,{G},1,2,0,1,2,2,2.0,5,51,2,"Hand(5 cards: Defiant Elf, Heedless One, Elven...","Graveyard(2 cards: Elvish Pioneer, Tribal Forc..."
3,Steven Guerrero,elvish rage,{G},1,3,0,1,5,3,3.0,2,50,5,"Hand(2 cards: Heedless One, Elven Riders)","Graveyard(5 cards: Elvish Pioneer, Tribal Forc..."
4,Steven Guerrero,elvish rage,{G},1,4,0,0,6,3,3.0,2,49,6,"Hand(2 cards: Heedless One, Elven Riders)","Graveyard(6 cards: Elvish Pioneer, Tribal Forc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Jessica Smith,sliver evolution,"{G, W, R}",10,1,0,1,0,1,0,7,52,0,"Hand(7 cards: Gemstone Mine, Mountain, Karplus...",Graveyard(0 cards: )
316,Jessica Smith,sliver evolution,"{G, W, R}",10,2,0,1,1,2,2.0,6,51,1,"Hand(6 cards: Mountain, Karplusan Forest, Plai...",Graveyard(1 cards: Two-Headed Sliver)
317,Jessica Smith,sliver evolution,"{G, W, R}",10,3,0,2,2,4,3.0,4,50,2,"Hand(4 cards: Plains, Karplusan Forest, Bonesp...","Graveyard(2 cards: Two-Headed Sliver, Harmonic..."
318,Jessica Smith,sliver evolution,"{G, W, R}",10,4,0,1,4,5,5.0,2,49,4,"Hand(2 cards: Karplusan Forest, Fury Sliver)","Graveyard(4 cards: Two-Headed Sliver, Harmonic..."


In [7]:
pd.set_option('display.max_columns', None)

def feature_engineering(matches_df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza engenharia de features nos dados das partidas de Magic: The Gathering.

    Esta função cria variáveis cumulativas e baseadas em razões a partir dos dados das partidas, e prepara
    o dataset para modelagem. Além disso, realiza a codificação de variáveis categóricas e trata valores
    ausentes ou infinitos, garantindo que os dados estejam adequados para uso em modelos preditivos.

    Args:
        matches_df (pd.DataFrame): DataFrame contendo os dados das partidas, com colunas como 'mana_pool', 
        'spent_mana', 'spells_played', 'lands_played', 'turn', e 'deck_colors', entre outras.

    Retorna:
        pd.DataFrame: DataFrame com novas features calculadas, incluindo:
        
        - 'cum_mana_pool': Soma cumulativa do mana disponível ao longo dos turnos para cada jogador e partida.
        - 'cum_spent_mana': Soma cumulativa do mana gasto ao longo dos turnos para cada jogador e partida.
        - 'cum_spells_played': Soma cumulativa dos feitiços jogados ao longo dos turnos para cada jogador e partida.
        - 'spell_ratio': Razão entre os feitiços jogados e o número de turnos (feitiços por turno).
        - 'land_ratio': Razão entre os terrenos jogados e o número de turnos (terrenos por turno).
        - 'mana_curve_efficiency': Eficiência da curva de mana, definida como a razão entre o mana gasto 
          cumulativo e o mana acumulado ao longo dos turnos.
        - Colunas binárias codificadas em One-Hot para representar as cores de mana ('W', 'U', 'B', 'R', 'G').

    A função também lida com erros de divisão, substituindo valores infinitos por 0, e preenche valores 
    ausentes na variável 'mana_curve_efficiency'.
    """

    # Configura o logger geral
    logger = setup_logger("feature_engineering")

    logger.info("Criando variáveis cumulativas por jogador e partida...")

    # Garantir que 'spent_mana' esteja no formato correto
    matches_df["spent_mana"] = matches_df["spent_mana"].astype(int)

    # Criação de variáveis cumulativas por 'name' e 'match'
    matches_df['cum_mana_pool'] = matches_df.groupby(['name', 'match'])['mana_pool'].cumsum()
    matches_df["cum_spent_mana"] = matches_df.groupby(['name', 'match'])["spent_mana"].cumsum()

    logger.info("Criando variáveis de razão...")

    # Criação de variáveis baseadas em razões: feitiços por turno e terrenos por turno
    matches_df['spell_ratio'] = (matches_df['spells_played'] / (matches_df['turn'] + 1)).round(2)
    matches_df['land_ratio'] = (matches_df['lands_played'] / (matches_df['turn'] + 1)).round(2)

    logger.info("Criando variável de eficiência da curva de mana...")

    # Criação da variável de eficiência da curva de mana (razão entre mana gasto e mana acumulado)
    matches_df['mana_curve_efficiency'] = matches_df['cum_spent_mana'] / matches_df['cum_mana_pool']

    # Tratamento de valores infinitos e valores ausentes
    matches_df['mana_curve_efficiency'].replace([float('inf'), -float('inf')], 0, inplace=True)
    matches_df['mana_curve_efficiency'].fillna(0, inplace=True)
    matches_df['mana_curve_efficiency'] = matches_df['mana_curve_efficiency'].round(2)

    # Codificação One-Hot para as cores de mana
    all_colors = ['W', 'U', 'B', 'R', 'G']

    logger.info("Codificando cores de mana (One-Hot Encoding)...")

    # Criar uma coluna binária para cada cor de mana
    for color in all_colors:
        matches_df[f'{color}'] = (matches_df['deck_colors'].apply(lambda x: 1 if color in x else 0)).astype("category")

    # Remover a coluna original de cores do deck
    matches_df.drop(columns=['deck_colors'], inplace=True)

    # Cria uma coluna com a quantidade de cores total do deck
    matches_df['n_colors'] = matches_df[all_colors].sum(axis=1)

    logger.info("Engenharia de features concluída.")

    return matches_df

# Chamada da função
features_df = catalog.load("matches_df")
features_df = feature_engineering(features_df)
catalog.save("features_df",features_df)
features_df

                    INFO     Loading data from matches_df (ParquetDataset)...                   ]8;id=574809;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=156541;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

2024-09-15 19:26:45,143 - INFO - Criando variáveis cumulativas por jogador e partida...
2024-09-15 19:26:45,146 - INFO - Criando variáveis de razão...
2024-09-15 19:26:45,148 - INFO - Criando variável de eficiência da curva de mana...
2024-09-15 19:26:45,149 - INFO - Codificando cores de mana (One-Hot Encoding)...
2024-09-15 19:26:45,162 - INFO - Engenharia de features concluída.


                    INFO     Saving data to features_df (ParquetDataset)...                     ]8;id=626283;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=388830;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

,name,deck_name,match,turn,mulligan_count,lands_played,spells_played,mana_pool,spent_mana,hand_size,library_size,graveyard_size,full_hand,full_graveyard,cum_mana_pool,cum_spent_mana,spell_ratio,land_ratio,mana_curve_efficiency,W,U,B,R,G,n_colors
0,Steven Guerrero,elvish rage,1,0,0,0,0,0,0,7,53,0,"Hand(7 cards: Karplusan Forest, Karplusan Fore...",Graveyard(0 cards: ),0,0,0.00,0.00,0.00,0,0,0,0,1,1
1,Steven Guerrero,elvish rage,1,1,0,1,1,1,1,6,52,1,"Hand(6 cards: Karplusan Forest, Tribal Forcema...",Graveyard(1 cards: Elvish Pioneer),1,1,0.50,0.50,1.00,0,0,0,0,1,1
2,Steven Guerrero,elvish rage,1,2,0,1,2,2,2,5,51,2,"Hand(5 cards: Defiant Elf, Heedless One, Elven...","Graveyard(2 cards: Elvish Pioneer, Tribal Forc...",3,3,0.67,0.33,1.00,0,0,0,0,1,1
3,Steven Guerrero,elvish rage,1,3,0,1,5,3,3,2,50,5,"Hand(2 cards: Heedless One, Elven Riders)","Graveyard(5 cards: Elvish Pioneer, Tribal Forc...",6,6,1.25,0.25,1.00,0,0,0,0,1,1
4,Steven Guerrero,elvish rage,1,4,0,0,6,3,3,2,49,6,"Hand(2 cards: Heedless One, Elven Riders)","Graveyard(6 cards: Elvish Pioneer, Tribal Forc...",9,9,1.20,0.00,1.00,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Jessica Smith,sliver evolution,10,1,0,1,0,1,0,7,52,0,"Hand(7 cards: Gemstone Mine, Mountain, Karplus...",Graveyard(0 cards: ),1,2,0.00,0.50,2.00,1,0,0,1,1,3
316,Jessica Smith,sliver evolution,10,2,0,1,1,2,2,6,51,1,"Hand(6 cards: Mountain, Karplusan Forest, Plai...",Graveyard(1 cards: Two-Headed Sliver),3,4,0.33,0.33,1.33,1,0,0,1,1,3
317,Jessica Smith,sliver evolution,10,3,0,2,2,4,3,4,50,2,"Hand(4 cards: Plains, Karplusan Forest, Bonesp...","Graveyard(2 cards: Two-Headed Sliver, Harmonic...",7,7,0.50,0.50,1.00,1,0,0,1,1,3
318,Jessica Smith,sliver evolution,10,4,0,1,4,5,5,2,49,4,"Hand(2 cards: Karplusan Forest, Fury Sliver)","Graveyard(4 cards: Two-Headed Sliver, Harmonic...",12,12,0.80,0.20,1.00,1,0,0,1,1,3


In [8]:
def feature_selection(
        features_df: pd.DataFrame,
        target: str,
        threshold_features: float,
        derived_features: list = None,
        key_columns: list = None) -> pd.DataFrame:
    """
    Auxilia na seleção de features, removendo aquelas derivadas do target, 
    colunas-chave, e features altamente correlacionadas entre si.

    Args:
        features_df (pd.DataFrame): DataFrame contendo as features.
        target (str): Nome da variável alvo.
        threshold_features (float): Limiar para remover features com alta correlação.
        derived_features (list, optional): Lista de features derivadas do target para serem removidas.
        key_columns (list, optional): Lista de colunas-chave para serem removidas (ex: 'match', 'turn').

    Returns:
        pd.DataFrame: DataFrame com as features selecionadas.
    """
    # Configura o logger geral
    logger = setup_logger("feature_selection")
    logger.info("Iniciando o processo de seleção de features...")

    # Separar apenas colunas numéricas para o cálculo da correlação
    numeric_features_df = features_df.select_dtypes(include=[np.number])

    logger.info(f"Número inicial de features numéricas: {numeric_features_df.shape[1]}")

    # Remover features derivadas do target, se fornecidas
    if derived_features:
        numeric_features_df = numeric_features_df.drop(columns=derived_features, errors='ignore')
        logger.info(f"Features derivadas do target removidas: {derived_features}")

    # Remover colunas-chave nao categoricas ou string
    if key_columns:
        numeric_features_df = numeric_features_df.drop(columns=key_columns, errors='ignore')
        logger.info(f"Colunas-chave removidas: {key_columns}")

    # Remover a variável alvo do conjunto de features
    features_without_target = numeric_features_df.drop(columns=[target], errors='ignore')

    # Calcular a matriz de correlação entre as features (excluindo o target)
    corr_matrix = features_without_target.corr().abs()

    # Criar uma máscara para identificar as correlações acima do limiar entre as features, excluindo a diagonal
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Identificar colunas com alta correlação entre si, usando o limiar definido
    to_drop_features = [column for column in upper.columns if any(upper[column] > threshold_features)]

    # Remover as colunas altamente correlacionadas entre si
    features_cleaned = features_without_target.drop(columns=to_drop_features)

    logger.info(f"Número de features após a remoção de correlação maior que {threshold_features}: {features_cleaned.shape[1]}")

    # Reconstruir o DataFrame final, reinserindo key_columns e target
    features_cleaned = pd.concat([features_cleaned, features_df[key_columns], features_df[[target]]], axis=1)

    logger.info("Processo de seleção de features concluído.")

    return features_cleaned


derived_features = [
    "cum_spent_mana", 
    "cum_mana_pool", 
    "spent_mana", 
    "mana_pool"
]

key_cols = [
    "name",
    "deck_name",
    "match",
    "turn"
]

features_df = catalog.load("features_df")
threshold_features = catalog.load("params:modeling.feature_engineering.feat_corr_threshold")

selected_features_df = feature_selection(
    features_df=features_df,         
    target="mana_curve_efficiency", 
    threshold_features=threshold_features,        
    derived_features=derived_features, 
    key_columns=key_cols            
)

# Salvando o DataFrame de features selecionadas
catalog.save("selected_features_df", selected_features_df)

                    INFO     Loading data from features_df (ParquetDataset)...                  ]8;id=65444;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=95352;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from                                                  ]8;id=982667;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=817220;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\
                             params:modeling.feature_engineering.feat_corr_threshold                               
                             (MemoryDataset)...                                                                    

2024-09-15 19:26:45,229 - INFO - Iniciando o processo de seleção de features...
2024-09-15 19:26:45,230 - INFO - Número inicial de features numéricas: 21
2024-09-15 19:26:45,231 - INFO - Features derivadas do target removidas: ['cum_spent_mana', 'cum_mana_pool', 'spent_mana', 'mana_pool']
2024-09-15 19:26:45,232 - INFO - Colunas-chave removidas: ['name', 'deck_name', 'match', 'turn']
2024-09-15 19:26:45,235 - INFO - Número de features após a remoção de correlação maior que 0.9: 11
2024-09-15 19:26:45,236 - INFO - Processo de seleção de features concluído.


                    INFO     Saving data to selected_features_df (ParquetDataset)...            ]8;id=992744;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=692074;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [9]:
selected_features_df

,mulligan_count,lands_played,spells_played,hand_size,spell_ratio,land_ratio,W,U,B,R,G,name,deck_name,match,turn,mana_curve_efficiency
0,0,0,0,7,0.00,0.00,0,0,0,0,1,Steven Guerrero,elvish rage,1,0,0.00
1,0,1,1,6,0.50,0.50,0,0,0,0,1,Steven Guerrero,elvish rage,1,1,1.00
2,0,1,2,5,0.67,0.33,0,0,0,0,1,Steven Guerrero,elvish rage,1,2,1.00
3,0,1,5,2,1.25,0.25,0,0,0,0,1,Steven Guerrero,elvish rage,1,3,1.00
4,0,0,6,2,1.20,0.00,0,0,0,0,1,Steven Guerrero,elvish rage,1,4,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,0,1,0,7,0.00,0.50,1,0,0,1,1,Jessica Smith,sliver evolution,10,1,2.00
316,0,1,1,6,0.33,0.33,1,0,0,1,1,Jessica Smith,sliver evolution,10,2,1.33
317,0,2,2,4,0.50,0.50,1,0,0,1,1,Jessica Smith,sliver evolution,10,3,1.00
318,0,1,4,2,0.80,0.20,1,0,0,1,1,Jessica Smith,sliver evolution,10,4,1.00


In [11]:
import random
from typing import List
import pandas as pd

def train_test_split(
        features_df: pd.DataFrame,
        final_features_list: List[str], 
        target_column: str, 
        n_test_players: int) -> None:
    """
    Segrega as partidas em treino e teste, permitindo que o modelo nunca veja um determinado grupo de jogadores durante o treino.
    Filtra o DataFrame pelas features selecionadas e produz os DataFrames de treino e teste para features e targets.

    Args:
        features_df (pd.DataFrame): O DataFrame contendo os dados das partidas.
        final_features_list (List[str]): Lista das features selecionadas para o modelo.
        target_column (str): Nome da coluna target.
        n_test_players (int): Número de jogadores a serem amostrados aleatoriamente para o conjunto de teste.
        catalog (Kedro catalog): O catálogo do Kedro, para salvar os DataFrames de treino e teste.

    Retorna:
        None: Salva os DataFrames de treino e teste (features e targets) no catálogo fornecido.
    """
    
    # Verifica se a quantidade de jogadores para o teste é válida
    unique_players = features_df['name'].unique()
    if n_test_players > len(unique_players):
        raise ValueError(f"O número de jogadores de teste ({n_test_players}) excede o número de jogadores únicos ({len(unique_players)}).")
    
    # Amostrando jogadores aleatoriamente
    test_players = random.sample(list(unique_players), n_test_players)
    print(f"Jogadores selecionados para o conjunto de teste: {test_players}")
    
    # Segregar os dados entre treino e teste com base nos jogadores amostrados
    test_df = features_df[features_df['name'].isin(test_players)]
    train_df = features_df[~features_df['name'].isin(test_players)]

    # Filtrando apenas as features selecionadas
    train_features = train_df[final_features_list]
    test_features = test_df[final_features_list]

    # Extraindo os targets
    train_target = pd.DataFrame(train_df[target_column])
    test_target = pd.DataFrame(test_df[target_column])


    return train_features, test_features, train_target, test_target


(train_features, test_features, train_target, test_target) = train_test_split(
    features_df=selected_features_df,
    final_features_list=selected_features_df.columns.tolist(),
    target_column="mana_curve_efficiency",
    n_test_players=1,
)

# Salvando as features e targets no catálogo
catalog.save("train_features", train_features)
catalog.save("test_features", test_features)
catalog.save("train_target", train_target)
catalog.save("test_target", test_target)

Jogadores selecionados para o conjunto de teste: ['Steven Schwartz']


[09/15/24 19:27:37] INFO     Saving data to train_features (ParquetDataset)...                  ]8;id=364020;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=829774;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

                    INFO     Saving data to test_features (ParquetDataset)...                   ]8;id=402700;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=555774;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

                    INFO     Saving data to train_target (ParquetDataset)...                    ]8;id=560924;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=383337;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

                    INFO     Saving data to test_target (ParquetDataset)...                     ]8;id=66681;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=279584;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\